In [1]:
import random
import math
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp2d
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

# 定义偶极子类
class Dipole:
    def __init__(self, place, orientation, weight):
        self.place=place
        self.orientation=orientation
        self.weight=weight
        pass

    def jx(self):
        return self.weight*self.orientation[0]/(self.orientation[0]**2+self.orientation[1]**2+self.orientation[2]**2)**0.5
    
    def jy(self):
        return self.weight*self.orientation[1]/(self.orientation[0]**2+self.orientation[1]**2+self.orientation[2]**2)**0.5
    
    def jz(self):
        return self.weight*self.orientation[2]/(self.orientation[0]**2+self.orientation[1]**2+self.orientation[2]**2)**0.5
    
    def __repr__(self):
        return 'dipole at (%f,%f,%f) with an orientation of (%f,%f,%f) and a weight of %f'%(self.place[0],self.place[1],self.place[2],self.orientation[0],self.orientation[1],self.orientation[2],self.weight)
        
    pass

# 定义全部场点平面位置
def findfieldpoints(a,n):
    fieldpoints=[]
    for i in range(n+1):
        for j in range(n+1):
            fieldpoints.append([(a/n)*i,(a/n)*j])
    return fieldpoints

# 定义探头位置
def findsensors(a,n,startpoint):
    sensorpoints=[]
    for i in range(n+1):
        for j in range(n+1):
            sensorpoints.append([startpoint[0]+(a/n)*i,startpoint[1]+(a/n)*j])
    return sensorpoints

# 联合探头高度，再次定义探头位置
def findtruesensorpoints(sensorpoints,h):
    truesensorpoints=[]
    for sensorpoint in sensorpoints:
        truesensorpoints.append(sensorpoint+[h])
    return truesensorpoints

# 定义需要计算的场点位置
def findtruefieldpoints(fieldpoints,truesensorpoints,shape,p1,c,n):
    truefieldpoints=[]
    if shape=='cubic':
        for truesensorpoint in truesensorpoints:
            unittruefieldpoints=[]
            for fieldpoint in fieldpoints:
                if abs(fieldpoint[0]-truesensorpoint[0]) <= p1/2 and abs(fieldpoint[1]-truesensorpoint[1]) <= p1/2:
                    for i in range(n): 
                        unittruefieldpoints.append(fieldpoint+[truesensorpoint[2]+(c/n)*i])
                else:
                    continue
            truefieldpoints.append(unittruefieldpoints)
        return truefieldpoints
    else:
        pass
    
# 计算
def calculate(dipoles,truefieldpoints):
    results=[]
    for unittruefieldpoints in truefieldpoints:
        foraverage=0
        count=0
        for truefieldpoint in unittruefieldpoints:
            count+=1
            unitforaverage=0
            for dipole in dipoles:
                unitforaverage+=(dipole.jx()*(truefieldpoint[1]-dipole.place[1])+dipole.jy()*(truefieldpoint[0]-dipole.place[0]))/((truefieldpoint[0]-dipole.place[0])**2+(truefieldpoint[1]-dipole.place[1])**2+(truefieldpoint[2]-dipole.place[2])**2)**1.5
            foraverage+=unitforaverage
        results.append(foraverage/count)
    return results

In [2]:
alldipoles = np.load(r'E:\sensorsize\alldipoles-10000.npy', allow_pickle=True)
allnoises = np.load(r'E:\sensorsize\allnoises-10000.npy', allow_pickle=True)

In [3]:
save_folder_11 =r"E:\sensorsize\testing36\10000_cubic_50_2.5_0_10_null_250_10-14"
save_folder_21 =r"E:\sensorsize\testing36\10000_cubic_50_2.5_10max_10_10max_250_10-14"

In [ ]:
# 生成所有场点位置
fieldpoints=findfieldpoints(24,240)

# 生成探头位置
sensorpoints=findsensors(20,5,[2,2])

# 生成含高度的探头位置
truesensorpoints=findtruesensorpoints(sensorpoints,2.5)

# 生成需要计算的场点位置
truefieldpoints1=findtruefieldpoints(fieldpoints,truesensorpoints,'cubic',5,5,50)

dipoles1=[]
dipoles1.append(Dipole([12,12,0],[1,0,0],10))

for pic_num in range(2485,10000):
    dipoles=alldipoles[pic_num]
    noises=allnoises[pic_num]

    # 生成偶极子
    dipoles2=[dipole for dipole in dipoles1]
    for i in range(len(dipoles)):
        dipoles2.append(Dipole([dipoles[i][0],dipoles[i][1],dipoles[i][2]],[dipoles[i][3],dipoles[i][4],dipoles[i][5]],dipoles[i][6])) 


    result11=calculate(dipoles1,truefieldpoints1)
    result21=calculate(dipoles2,truefieldpoints1)


    # 显示结果
    x = np.array([sensorpoint[0] for sensorpoint in sensorpoints])
    y = np.array([sensorpoint[1] for sensorpoint in sensorpoints])
    Z11 = np.array(result11)
    Z21 = np.array(result21)

    Z11 += noises
    Z21 += noises
    
    x2 = np.linspace(0, 24, 128)
    y2 = np.linspace(0, 24, 128)
    f11 = interp2d(x, y, Z11, kind='cubic')
    f21 = interp2d(x, y, Z21, kind='cubic')
    
    Z211 = f11(x2, y2)
    np.save(save_folder_11+'\\'+ str(pic_num),Z211)
    Z221 = f21(x2, y2)
    np.save(save_folder_21+'\\'+ str(pic_num),Z221)

c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000100 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000097 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too

c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000127 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000049 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too

c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000138 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000040 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too

c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000210 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000211 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too

c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000103 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
c:\program files\python37\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,12 m=36 fp=0.000120 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))
